In [13]:
import pandas as pd
import numpy as np
from joblib import load, dump
from copy import deepcopy
from statistics import mean

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score, recall_score
from collections import Counter

In [14]:
import mlfoundry as mlf

# from getpass import getpass
# api_token = getpass("TrueFoundry API Token:")

In [15]:
mlf.login()

API key is already configured.
Please use `mlfoundry login --relogin` or `mlfoundry.login(relogin=True)`to force relogin


False

In [16]:
# import pdb

In [17]:
# try:
#     mlf_api = mlf.get_client()
# except Exception:
#     pdb.post_mortem()


In [18]:
mlf_api = mlf.get_client()

In [19]:
import os
os.getenv('MLF_HOST')

In [20]:
df = pd.read_csv('data/Features.csv')

In [21]:
df.head()

,product,answer_option,label,review_len,Rn,Rp,Rs,Rc,Rd,Rsc
0,Accucheck,Fast and accurate delivery,0,4,0.232859,0.3,0.616667,0.005420,1.0,0.0000
1,Accucheck,Expected a longer expire date. Your Product Li...,0,14,0.596318,-0.1,0.400000,0.017615,1.0,0.0000
2,Accucheck,I liked the prompt service,0,5,0.319747,0.6,0.800000,0.006775,1.0,0.4215
3,Accucheck,Good product,0,2,0.546925,0.7,0.600000,0.002710,0.0,0.4404
4,Accucheck,I not needed,0,3,0.000000,0.0,0.000000,0.004065,0.0,0.0000


In [22]:
data_split = pd.crosstab(df['product'],df['label'])
data_split.head()

label,0,1
product,,
Accucheck,310,85
Becadexamin,53,27
Evion,89,33
Neurobion,280,136
SevenseascodLiverOil,59,22


In [23]:
def building_training_data(df):
    A = df[df['label']==1]
    A.loc[df['label']==1,'join'] = 'j'
    B = df[df['label']==0]
    B.loc[df['label']==0,'join'] = 'j'
    trainset1 = pd.merge(A,B,how='outer',on='join')
    trainset2 = pd.merge(B,A,how='outer',on ='join')

    trainset = pd.merge(trainset1,trainset2,how='outer')
    return trainset

In [24]:
product_list = df['product'].unique()
data_stack = []
for product in product_list:
    temp = deepcopy(df[df['product']==product].iloc[:,2:])
    build_data = building_training_data(temp)
    print(product, len(temp), len(build_data))
    build_data.drop(columns = ['join','label_y'],inplace=True)
    data = build_data.iloc[:,1:]
    data['target'] = build_data.iloc[:,0]
    data_stack.append(data)

e:\true_faundry\projectpro-external-main\Projects\venv\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
e:\true_faundry\projectpro-external-main\Projects\venv\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Accucheck 395 52700
Becadexamin 80 2862
Evion 122 5874
Neurobion 416 76160
SevenseascodLiverOil 81 2596
Shelcal 383 64232
Supradyn 73 2300
shampoo 105 5488


In [25]:
train = pd.concat(data_stack).reset_index(drop = True)

In [26]:
train

,review_len_x,Rn_x,Rp_x,Rs_x,Rc_x,Rd_x,Rsc_x,review_len_y,Rn_y,Rp_y,Rs_y,Rc_y,Rd_y,Rsc_y,target
0,5,0.544357,0.52,0.823333,0.006775,0.0,0.0000,4,0.232859,0.300000,0.616667,0.005420,1.0,0.0000,1
1,5,0.544357,0.52,0.823333,0.006775,0.0,0.0000,14,0.596318,-0.100000,0.400000,0.017615,1.0,0.0000,1
2,5,0.544357,0.52,0.823333,0.006775,0.0,0.0000,5,0.319747,0.600000,0.800000,0.006775,1.0,0.4215,1
3,5,0.544357,0.52,0.823333,0.006775,0.0,0.0000,2,0.546925,0.700000,0.600000,0.002710,0.0,0.4404,1
4,5,0.544357,0.52,0.823333,0.006775,0.0,0.0000,2,0.546925,0.700000,0.600000,0.002710,0.0,0.4404,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212207,10,0.413633,0.50,0.500000,0.023866,0.0,-0.1263,15,0.571960,0.700000,0.600000,0.035800,0.0,0.1531,0
212208,10,0.413633,0.50,0.500000,0.023866,0.0,-0.1263,7,0.392391,0.316667,0.600000,0.016706,0.0,0.2382,0
212209,10,0.413633,0.50,0.500000,0.023866,0.0,-0.1263,25,0.575335,0.233333,0.255556,0.059666,0.0,0.5927,0
212210,10,0.413633,0.50,0.500000,0.023866,0.0,-0.1263,10,0.472342,0.500000,0.500000,0.023866,0.0,0.1779,0


In [27]:
X = train.iloc[:,:-1]
y = train.iloc[:,-1]

from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2,shuffle = True, stratify = y) 
print("Test Len:",len(X_test)," ",len(y_test))

Test Len: 42443   42443


In [28]:
train

,review_len_x,Rn_x,Rp_x,Rs_x,Rc_x,Rd_x,Rsc_x,review_len_y,Rn_y,Rp_y,Rs_y,Rc_y,Rd_y,Rsc_y,target
0,5,0.544357,0.52,0.823333,0.006775,0.0,0.0000,4,0.232859,0.300000,0.616667,0.005420,1.0,0.0000,1
1,5,0.544357,0.52,0.823333,0.006775,0.0,0.0000,14,0.596318,-0.100000,0.400000,0.017615,1.0,0.0000,1
2,5,0.544357,0.52,0.823333,0.006775,0.0,0.0000,5,0.319747,0.600000,0.800000,0.006775,1.0,0.4215,1
3,5,0.544357,0.52,0.823333,0.006775,0.0,0.0000,2,0.546925,0.700000,0.600000,0.002710,0.0,0.4404,1
4,5,0.544357,0.52,0.823333,0.006775,0.0,0.0000,2,0.546925,0.700000,0.600000,0.002710,0.0,0.4404,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212207,10,0.413633,0.50,0.500000,0.023866,0.0,-0.1263,15,0.571960,0.700000,0.600000,0.035800,0.0,0.1531,0
212208,10,0.413633,0.50,0.500000,0.023866,0.0,-0.1263,7,0.392391,0.316667,0.600000,0.016706,0.0,0.2382,0
212209,10,0.413633,0.50,0.500000,0.023866,0.0,-0.1263,25,0.575335,0.233333,0.255556,0.059666,0.0,0.5927,0
212210,10,0.413633,0.50,0.500000,0.023866,0.0,-0.1263,10,0.472342,0.500000,0.500000,0.023866,0.0,0.1779,0


In [29]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train,y_train)
print("Training Accuracy\n", accuracy_score(y_train,classifier.predict(X_train)))
print("Test Accuracy\n", accuracy_score(y_test,classifier.predict(X_test)))

print('CLASSIFICATION REPORT')
print("Training\n", classification_report(y_train,classifier.predict(X_train)))
print("Test \n", classification_report(y_test,classifier.predict(X_test)))

e:\true_faundry\projectpro-external-main\Projects\venv\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


Training Accuracy
 0.8511035583646013
Test Accuracy
 0.8501284075112504
CLASSIFICATION REPORT
Training
               precision    recall  f1-score   support

           0       0.85      0.85      0.85     84885
           1       0.85      0.85      0.85     84884

    accuracy                           0.85    169769
   macro avg       0.85      0.85      0.85    169769
weighted avg       0.85      0.85      0.85    169769

Test 
               precision    recall  f1-score   support

           0       0.85      0.85      0.85     21221
           1       0.85      0.85      0.85     21222

    accuracy                           0.85     42443
   macro avg       0.85      0.85      0.85     42443
weighted avg       0.85      0.85      0.85     42443



In [30]:
mlf_run = mlf_api.create_run(project_name='ecommerce-product-reviews', run_name='Logistic-Regression')

train_dataset = X_train.copy()
print(train_dataset.index)
train_dataset['targets'] = y_train
train_dataset['predictions'] = classifier.predict(X_train)
train_dataset['prediction_probabilities'] = list(classifier.predict_proba(X_train))


[mlfoundry] 2022-06-30T15:39:02+0530 INFO project ecommerce-product-reviews does not exist. Creating ecommerce-product-reviews.
Link to the dashboard for the run: https://app.truefoundry.com/mlfoundry/147/7f838202516f487fb1f3b74a494a4f6d/
[mlfoundry] 2022-06-30T15:39:09+0530 WARNING failed to log git info because git repository is not present
[mlfoundry] 2022-06-30T15:39:11+0530 INFO Run 'Kumar/ecommerce-product-reviews/Logistic-Regression' has started.
Int64Index([205377, 167395,  23784,  57842, 155061,  38796,   4592, 193064,
            205211, 189686,
            ...
            130800,  23694,  56660, 161721, 163474, 202015, 172215, 140770,
             49967,  26165],
           dtype='int64', length=169769)


In [31]:
mlf_run.log_dataset(
    dataset_name = 'train_dataset',
    features = train_dataset[list(X_train.columns)],
    predictions = train_dataset['predictions'],
    actuals = train_dataset['targets'],
    only_stats = False,   
)

[mlfoundry] 2022-06-30T15:39:12+0530 INFO Logging Dataset, this might take a while ...


About to log a dataframe with 2376766 elements, logging might take some time to complete.


[mlfoundry] 2022-06-30T15:42:26+0530 INFO Dataset logged successfully
To visualize the logged dataset, click on the link https://app.truefoundry.com/mlfoundry/147/7f838202516f487fb1f3b74a494a4f6d/?tab=data-feature-metrics


In [32]:
test_dataset = X_test.copy()
test_dataset['targets'] = y_test
test_dataset['predictions'] = classifier.predict(X_test)
test_dataset['prediction_probabilities'] = list(classifier.predict_proba(X_test))

In [33]:
mlf_run.log_dataset(
    dataset_name = 'test_dataset',
    features = test_dataset[list(X_test.columns)],
    predictions = test_dataset['predictions'],
    actuals = test_dataset['targets'],
    only_stats = False,
)

[mlfoundry] 2022-06-30T15:42:27+0530 INFO Logging Dataset, this might take a while ...


About to log a dataframe with 594202 elements, logging might take some time to complete.


[mlfoundry] 2022-06-30T15:43:40+0530 INFO Dataset logged successfully
To visualize the logged dataset, click on the link https://app.truefoundry.com/mlfoundry/147/7f838202516f487fb1f3b74a494a4f6d/?tab=data-feature-metrics


In [34]:
y_predict = classifier.predict(X_test)

metrics_dict = {
    "Accuracy": accuracy_score(y_test, y_predict),
    "Precision": precision_score(y_test, y_predict),
    "Recall": recall_score(y_test, y_predict),
}

mlf_run.log_metrics(metrics_dict)

[mlfoundry] 2022-06-30T15:43:42+0530 INFO Metrics logged successfully


In [35]:
mlf_run.log_model(classifier, framework=mlf.ModelFramework.SKLEARN)
mlf_run.log_params(classifier.get_params())

[mlfoundry] 2022-06-30T15:44:37+0530 INFO Model logged successfully
[mlfoundry] 2022-06-30T15:44:39+0530 INFO Parameters logged successfully


In [36]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
classifier.fit(X_train,y_train)

print("Training Accuracy\n", accuracy_score(y_train,classifier.predict(X_train)))
print("Test Accuracy\n", accuracy_score(y_test,classifier.predict(X_test)))

print('CLASSIFICATION REPORT')
print("Training\n", classification_report(y_train,classifier.predict(X_train)))
print("Test \n", classification_report(y_test,classifier.predict(X_test)))

Training Accuracy
 0.9966071544274867
Test Accuracy
 0.9836957802228872
CLASSIFICATION REPORT
Training
               precision    recall  f1-score   support

           0       0.99      1.00      1.00     84885
           1       1.00      0.99      1.00     84884

    accuracy                           1.00    169769
   macro avg       1.00      1.00      1.00    169769
weighted avg       1.00      1.00      1.00    169769

Test 
               precision    recall  f1-score   support

           0       0.98      0.99      0.98     21221
           1       0.98      0.98      0.98     21222

    accuracy                           0.98     42443
   macro avg       0.98      0.98      0.98     42443
weighted avg       0.98      0.98      0.98     42443



In [37]:
mlf_run = mlf_api.create_run(project_name='ecommerce-product-reviews', run_name='Decision-Tree')

train_dataset = X_train.copy()
train_dataset['targets'] = y_train
train_dataset['predictions'] = classifier.predict(X_train)
train_dataset['prediction_probabilities'] = list(classifier.predict_proba(X_train))

Link to the dashboard for the run: https://app.truefoundry.com/mlfoundry/147/683081be838146e2a69b78b78e0724ca/
[mlfoundry] 2022-06-30T15:45:00+0530 WARNING failed to log git info because git repository is not present
[mlfoundry] 2022-06-30T15:45:03+0530 INFO Run 'Kumar/ecommerce-product-reviews/Decision-Tree' has started.


In [38]:
mlf_run.log_dataset(
    dataset_name = 'train_dataset',
    features = train_dataset[list(X_train.columns)],
    predictions = train_dataset['predictions'],
    actuals = train_dataset['targets'],
    only_stats = False,   
)

[mlfoundry] 2022-06-30T15:45:05+0530 INFO Logging Dataset, this might take a while ...


About to log a dataframe with 2376766 elements, logging might take some time to complete.


[mlfoundry] 2022-06-30T15:46:24+0530 INFO Shutting down background jobs and syncing data for run 'Kumar/ecommerce-product-reviews/Logistic-Regression', please don't kill this process...
[mlfoundry] 2022-06-30T15:46:24+0530 WARNING failed to set termination status FINISHED due to cannot join current thread
Link to the dashboard for the run: https://app.truefoundry.com/mlfoundry/147/7f838202516f487fb1f3b74a494a4f6d/
[mlfoundry] 2022-06-30T15:50:23+0530 INFO Dataset logged successfully
To visualize the logged dataset, click on the link https://app.truefoundry.com/mlfoundry/147/683081be838146e2a69b78b78e0724ca/?tab=data-feature-metrics


In [39]:
test_dataset = X_test.copy()
test_dataset['targets'] = y_test
test_dataset['predictions'] = classifier.predict(X_test)
test_dataset['prediction_probabilities'] = list(classifier.predict_proba(X_test))

In [40]:
mlf_run.log_dataset(
    dataset_name = 'test_dataset',
    features = test_dataset[list(X_test.columns)],
    predictions = test_dataset['predictions'],
    actuals = test_dataset['targets'],
    only_stats = False,
)

[mlfoundry] 2022-06-30T15:50:24+0530 INFO Logging Dataset, this might take a while ...


About to log a dataframe with 594202 elements, logging might take some time to complete.


[mlfoundry] 2022-06-30T15:52:19+0530 INFO Dataset logged successfully
To visualize the logged dataset, click on the link https://app.truefoundry.com/mlfoundry/147/683081be838146e2a69b78b78e0724ca/?tab=data-feature-metrics


In [41]:
y_predict = classifier.predict(X_test)

metrics_dict = {
    "Accuracy": accuracy_score(y_test, y_predict),
    "Precision": precision_score(y_test, y_predict),
    "Recall": recall_score(y_test, y_predict),
}

mlf_run.log_metrics(metrics_dict)

[mlfoundry] 2022-06-30T15:52:27+0530 INFO Metrics logged successfully


In [42]:
mlf_run.log_model(classifier, framework=mlf.ModelFramework.SKLEARN)
mlf_run.log_params(classifier.get_params())

[mlfoundry] 2022-06-30T15:53:15+0530 INFO Model logged successfully
[mlfoundry] 2022-06-30T15:53:17+0530 INFO Parameters logged successfully


In [43]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=50, n_jobs = -1, oob_score = True,random_state=42)
classifier.fit(X_train,y_train)

print("Training Accuracy\n", accuracy_score(y_train,classifier.predict(X_train)))
print("Test Accuracy\n", accuracy_score(y_test,classifier.predict(X_test)))

print('CLASSIFICATION REPORT')
print("Training\n", classification_report(y_train,classifier.predict(X_train)))
print("Test \n", classification_report(y_test,classifier.predict(X_test)))

print("Test\nConfusion Matrix: \n", confusion_matrix(y_test, classifier.predict(X_test)))

Training Accuracy
 0.9966071544274867
Test Accuracy
 0.9889970077515727
CLASSIFICATION REPORT
Training
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     84885
           1       1.00      1.00      1.00     84884

    accuracy                           1.00    169769
   macro avg       1.00      1.00      1.00    169769
weighted avg       1.00      1.00      1.00    169769

Test 
               precision    recall  f1-score   support

           0       0.99      0.99      0.99     21221
           1       0.99      0.99      0.99     21222

    accuracy                           0.99     42443
   macro avg       0.99      0.99      0.99     42443
weighted avg       0.99      0.99      0.99     42443

Test
Confusion Matrix: 
 [[20986   235]
 [  232 20990]]


In [44]:
## Score of the training dataset obtained using an out-of-bag estimate. This attribute exists only when oob_score is True.
classifier.oob_score_

0.9881073694255135

In [45]:
feature_importances = pd.DataFrame(classifier.feature_importances_,
                                   index = train.iloc[:,:-1].columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances

,importance
Rd_y,0.191935
Rd_x,0.182769
review_len_x,0.122598
review_len_y,0.100122
Rc_y,0.086639
Rc_x,0.066476
Rsc_x,0.041485
Rsc_y,0.038831
Rn_x,0.038809
Rn_y,0.038258


In [46]:
dump(classifier, 'randomforest.joblib', compress = 2)

['randomforest.joblib']

In [47]:
mlf_run = mlf_api.create_run(project_name='ecommerce-product-reviews', run_name='Random-Forest')

Link to the dashboard for the run: https://app.truefoundry.com/mlfoundry/147/bf74414fcbe94a7997831abfbda3e1f2/
[mlfoundry] 2022-06-30T15:55:17+0530 WARNING failed to log git info because git repository is not present
[mlfoundry] 2022-06-30T15:55:19+0530 INFO Run 'Kumar/ecommerce-product-reviews/Random-Forest' has started.
[mlfoundry] 2022-06-30T15:55:26+0530 INFO Shutting down background jobs and syncing data for run 'Kumar/ecommerce-product-reviews/Decision-Tree', please don't kill this process...
[mlfoundry] 2022-06-30T15:55:30+0530 INFO Finished syncing data for run 'Kumar/ecommerce-product-reviews/Decision-Tree'. Thank you for waiting!
Link to the dashboard for the run: https://app.truefoundry.com/mlfoundry/147/683081be838146e2a69b78b78e0724ca/


In [48]:
train_dataset = X_train.copy()
train_dataset['targets'] = y_train
train_dataset['predictions'] = classifier.predict(X_train)
train_dataset['prediction_probabilities'] = list(classifier.predict_proba(X_train))

In [49]:
mlf_run.log_dataset(
    dataset_name = 'train_dataset',
    features = train_dataset[list(X_train.columns)],
    predictions = train_dataset['predictions'],
    actuals = train_dataset['targets'],
    only_stats = False,   
)

[mlfoundry] 2022-06-30T15:55:43+0530 INFO Logging Dataset, this might take a while ...


About to log a dataframe with 2376766 elements, logging might take some time to complete.


[mlfoundry] 2022-06-30T16:00:52+0530 INFO Dataset logged successfully
To visualize the logged dataset, click on the link https://app.truefoundry.com/mlfoundry/147/bf74414fcbe94a7997831abfbda3e1f2/?tab=data-feature-metrics


In [50]:
test_dataset = X_test.copy()
test_dataset['targets'] = y_test
test_dataset['predictions'] = classifier.predict(X_test)
test_dataset['prediction_probabilities'] = list(classifier.predict_proba(X_test))

In [51]:
mlf_run.log_dataset(
    dataset_name = 'test_dataset',
    features = test_dataset[list(X_test.columns)],
    predictions = test_dataset['predictions'],
    actuals = test_dataset['targets'],
    only_stats = False,
)

[mlfoundry] 2022-06-30T16:00:59+0530 INFO Logging Dataset, this might take a while ...


About to log a dataframe with 594202 elements, logging might take some time to complete.


[mlfoundry] 2022-06-30T16:02:53+0530 INFO Dataset logged successfully
To visualize the logged dataset, click on the link https://app.truefoundry.com/mlfoundry/147/bf74414fcbe94a7997831abfbda3e1f2/?tab=data-feature-metrics


In [52]:
y_predict = classifier.predict(X_test)

metrics_dict = {
    "Accuracy": accuracy_score(y_test, y_predict),
    "Precision": precision_score(y_test, y_predict),
    "Recall": recall_score(y_test, y_predict),
}

mlf_run.log_metrics(metrics_dict)

[mlfoundry] 2022-06-30T16:03:02+0530 INFO Metrics logged successfully


In [53]:
mlf_run.log_model(classifier, framework=mlf.ModelFramework.SKLEARN)
mlf_run.log_params(classifier.get_params())

In [ ]:
classifier = load('randomforest.joblib')

In [ ]:
product_list = df['product'].unique()
df['win']=0
df['lose']=0
df['review_score'] = 0.0
df.reset_index(inplace = True, drop = True)


def score_giver(C,D):
    E = pd.merge(C,D,how='outer',on='j')
    E.drop(columns=['j'],inplace = True)
    q= classifier.predict(E.values)
    return Counter(q)

for product in product_list:
    data = df[df['product']==product]
    for indx in data.index:
        review = df.iloc[indx, 3:-3]
        review['j'] = 'jn'
        C = pd.DataFrame([review])
        D = data[data.index!=indx].iloc[:,3:-3]
        D['j'] = 'jn'
        score = score_giver(C,D)
        df.at[indx, 'win'] = 0 if score.get(1) is None else score.get(1)
        df.at[indx, 'lose'] = 0 if score.get(0) is None else score.get(0)
        df.at[indx, 'review_score'] = float(0 if score.get(1) is None else score.get(1)) / len(data) * 1.0

df = df.sort_values(by = ['product','review_score'], ascending = False)

r_accuracy =[]
for product in product_list:
    x = data_split[data_split.index == product][1][0]
    number_of_1_in_x = Counter(df[df['product']==product].iloc[:x, ]['label']).get(1)
    rank_accuracy = float(number_of_1_in_x*1.0 / x*1.0)
    print("Product: {} | Rank Accuracy: {}".format(product, rank_accuracy))
    r_accuracy.append(rank_accuracy)
print("Mean Rank Accuracy: {}".format(mean(r_accuracy)))

In [ ]:
df

In [ ]:
df.iloc[:, [0,1,-1]].to_csv('data/train_ranked_output.csv',index = False)

In [ ]:
# !ls

In [ ]:
t = pd.read_csv('data/test.csv')